## Classifier: In Context Learing

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

import numpy as np
import os
import time
import random
import torch

/home/pop529700/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # Limit visibility to only GPU 0
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# # Set the device to GPU 0 if available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Define Prompt Template:

In [3]:
SAMPLING = "random" # "random", "stratified", "clustered", "shared_domain"
SUFFIX = "_extended" #"", "_holdout", "_extended"
SPLIT = "test" # "train", "test", "holdout", "extende
MAX_CONTENT_LENGTH = 384 # 496, 192
OVERLAP = 64
FEATURES = "url_and_content" # "url", "content", "url_and_content"

In [4]:
DEMONSTR_SAMPLING = "random_balanced" 
K = 4

In [5]:
TOPICS = ["cannabis", "energie", "kinder"]
MODELS = [
    # {
    #     "name": "aya-101",
    #     "model": "CohereForAI/aya-101",
    #     "tokenizer_class": "AutoTokenizer",
    #     "model_class": "AutoModelForSeq2SeqLM"
    # },
    # {
    #     "name": "vicuna-13b",
    #     "model": "lmsys/vicuna-13b-v1.5",
    #     "tokenizer_class": "LlamaTokenizer",
    #     "model_class": "LlamaForCausalLM"
    # },
    # {
    #      "name": "vicuna-7b",
    #      "model": "lmsys/vicuna-7b-v1.5",
    #      "tokenizer_class": "LlamaTokenizer",
    #      "model_class": "LlamaForCausalLM"
    #  },
    #  {
    #      "name": "FLAN-t5-base",
    #      "model": "google/flan-t5-base",
    #      "tokenizer_class": "AutoTokenizer",
    #      "model_class": "AutoModelForSeq2SeqLM"
    #  },
    #  {
    #      "name": "FLAN-t5-large",
    #      "model": "google/flan-t5-large",
    #      "tokenizer_class": "AutoTokenizer",
    #      "model_class": "AutoModelForSeq2SeqLM"
    #  },
    #  {
    #      "name": "FLAN-t5-xxl",
    #         "model": "google/flan-t5-xxl",
    #      "tokenizer_class": "AutoTokenizer",
    #      "model_class": "AutoModelForSeq2SeqLM"
    #  },
    # {
    #     "name": "leo-hessianai-13b-chat",
    #     "model": "LeoLM/leo-hessianai-13b-chat",
    #     "tokenizer_class": "AutoTokenizer",
    #     "model_class": "AutoModelForCausalLM"
    # },
    # {
    #     "name": "leo-hessianai-7b-chat",
    #     "model": "LeoLM/leo-hessianai-7b-chat",
    #     "tokenizer_class": "AutoTokenizer",
    #     "model_class": "AutoModelForCausalLM"
    # },
    # {
    # "name": "leo-hessianai-7b-chat-bilingual",
    # "model": "LeoLM/leo-hessianai-7b-chat-bilingual",
    # "tokenizer_class": "AutoTokenizer",
    # "model_class": "AutoModelForCausalLM"
    # },
    {
    "name": "leo-mistral-hessianai-7b-chat",
    "model": "LeoLM/leo-mistral-hessianai-7b-chat",
    "tokenizer_class": "AutoTokenizer",
    "model_class": "AutoModelForCausalLM"
    },
    # {
    #     "name": "aya-23-8B",
    #     "model": "CohereForAI/aya-23-8B",
    #     "tokenizer_class": "AutoTokenizer",
    #     "model_class": "AutoModelForCausalLM"
    # },

]

## Define Prompt Template:

**Topic Desciptions:**

In [6]:
topic_desciptions = {
    "kinder": {
        "name": "Kindergrundsicherung",
        "description": "Die Kindergrundsicherung zielt darauf ab, Kinderarmut zu bekämpfen, indem sie einen festen Betrag, einkommensabhängige Zuschläge und Bildungsleistungen bereitstellt.",
        "keywords": ['kinder', 'kindergr', 'paus', 'familie', 'bundestag.de', 'arbeitsagentur.de', 'kindergrundsicherung',  'kindergeld', 'kindersicherung', 'kinderzuschlag', 'gesetz']
    },
    "cannabis": {
        "name": "die Legalisierung von Cannabis",
        "description": "Das CanG 2023 (Cannabisgesetz) wird den privaten Anbau von Cannabis durch Erwachsene für den persönlichen Gebrauch und den kollektiven nicht-kommerziellen Anbau legalisieren.",
        "keywords": ['cannabis', 'canabis', 'cannabic', 'gras', 'cbd' , 'droge', 'hanf', 'thc', 'canbe', 'legal', 'legalisierung', 'gesetz', 'verein', 'entkriminali']
    },
    "energie": {
        "name": "erneuerbare Energiequellen",
        "description": "Das EEG 2023 (Erneuerbare-Energien-Gesetz) zielt darauf ab, den Anteil erneuerbarer Energien am Bruttostromverbrauch bis 2030 auf mindestens 80% zu erhöhen.",
        "keywords": ['energie', 'eeg','grün','gruen','habeck', 'climate', 'strom','Waerme','wende','frderung', 'förderung', 'windkraft', 'windrad', 'photovoltaik',
                     'photovoltaic', 'solar', 'heizung', 'heiz', 'gesetz', 'erneuer', 'geothermie', 'pv', 'geg']
    }
}


**Prompt Templates:**

In [7]:
############################### ZERO SHOT ###############################

PROMPT_TEMPLATE_ZERO_SHOT = """Klassifiziere den folgenden Webseite-Text auf {lang} als themenbezogen oder nicht themenbezogen. Enthält er Informationen über '{topic}'? Bitte antworte nur mit 'Ja' oder 'Nein'.

Themenbeschreibung: {topic_desc}
Themen-Schlüsselwörter: {topic_keyw}

URL: '''{url}'''
Text: '''{webpage_text}'''

Enthält dieser Text relevante Informationen über '{topic}'?
"""

############################### FEW SHOT ###############################

PROMPT_TEMPLATE_FEW_SHOT = """Klassifiziere die folgende Webseite auf {lang} als themenbezogen oder nicht themenbezogen. Enthält er Informationen über '{topic}'? Bitte antworte nur mit 'Ja' oder 'Nein'.

Themenbeschreibung: {topic_desc}
Themen-Schlüsselwörter: {topic_keyw}

Beispiele:
{examples}

-------


Webseite :
URL: '''{url}'''
Text: '''{webpage_text}'''

Enthält diese Webseite relevante Informationen über '{topic}'?
"""

############################### DEMONSTRATOR ###############################

PROMPT_TEMPLATE_EXAMPLES = """
URL: '''{url}'''
Text: '''{text}'''
Antwort: '{label}'
"""

############################### System Prompt ###############################

SYTEM_PROMPT_TEMPLATE = """Du bist ein fortschrittliches Sprachmodell, das darauf ausgelegt ist, Texte entweder als "themenbezogen" oder "nicht themenbezogen" zu klassifizieren."""


**Test prompt templates:**

In [8]:

# Example text
example_list = [
    {
        "text": "Cannabis ist eine Droge.",
        "view_url": "google.de",
        "label": "Yes"
    },
    {
        "text": "Katzen sind Tiere.",
        "view_url": "example.com",
        "label": "No"
    }
]

example_prompt_list = [PROMPT_TEMPLATE_EXAMPLES.format(text=example["text"], url=example["view_url"],  label=example["label"]) for example in example_list]

# print("Example prompt list:")
# print("\n".join(example_prompt_list))

topic_desciption = topic_desciptions["cannabis"]
topic_name = topic_desciption.get("name")
topic_desc = topic_desciption.get("description")
topic_keyw = topic_desciption.get("keywords")


# Test the template with a dummy text
prompt_test = PROMPT_TEMPLATE_FEW_SHOT.format(topic = topic_name, lang = 'German', url = "example.com",  topic_desc = topic_desc, topic_keyw = ", ".join(topic_keyw),  webpage_text='Lorem ipsum dolor sit amet, consectetur adipiscing elit.', examples="".join(example_prompt_list))
print(prompt_test)

Klassifiziere die folgende Webseite auf German als themenbezogen oder nicht themenbezogen. Enthält er Informationen über 'die Legalisierung von Cannabis'? Bitte antworte nur mit 'Ja' oder 'Nein'.

Themenbeschreibung: Das CanG 2023 (Cannabisgesetz) wird den privaten Anbau von Cannabis durch Erwachsene für den persönlichen Gebrauch und den kollektiven nicht-kommerziellen Anbau legalisieren.
Themen-Schlüsselwörter: cannabis, canabis, cannabic, gras, cbd, droge, hanf, thc, canbe, legal, legalisierung, gesetz, verein, entkriminali

Beispiele:

URL: '''google.de'''
Text: '''Cannabis ist eine Droge.'''
Antwort: 'Yes'

URL: '''example.com'''
Text: '''Katzen sind Tiere.'''
Antwort: 'No'


-------


Webseite :
URL: '''example.com'''
Text: '''Lorem ipsum dolor sit amet, consectetur adipiscing elit.'''

Enthält diese Webseite relevante Informationen über 'die Legalisierung von Cannabis'?



In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("LeoLM/leo-hessianai-7b-chat", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:

def convert_prompt_to_chat(tokenizer, SYTEM_PROMPT_TEMPLATE, prompt):
    """
    Convert a prompt to a chat format
    """
    messages = [
        {"role": "system", "content": SYTEM_PROMPT_TEMPLATE},
        {"role": "user", "content": prompt}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    #return tokenizer.decode(input_ids[0])

chat_prompt = convert_prompt_to_chat(tokenizer, SYTEM_PROMPT_TEMPLATE, prompt_test)
print(chat_prompt)


tensor([[32005,  1788,    13, 29928, 29884,   289,   391,  1011,  5162,   816,
         12123, 20048,  1706, 10221,  1545,   514, 29892,  1697, 19118, 17691,
         16260,  1752, 29892,  8490,   371,   875, 29893,  2447,  1620,   376,
           386, 12398, 15325,  6352, 29908,  4461,   376, 29876,  1428,   963,
           264, 15325,  6352, 29908,  1729, 22902, 21722,  7884, 29889, 32006,
         29871,    13, 32005,  1404,    13, 29968,   605,   361, 10077,   406,
           762,   900, 11693,  2563, 11485,  1622,  5332,  1620,   963,   264,
         15325,  6352,  4461,  3072,   963,   264, 15325,  6352, 29889,  1174,
           386, 10905,   604, 10343,   264,  2939,   525, 16217,  5682,  5711,
          7263,  1005,   315,   812,   370,   275, 29915, 29973, 19064,  3677,
         29893, 10069,  5595,  1380,   525, 29967, 29874, 29915,  4461,   525,
          8139,   262,  4286,    13,    13,  1349, 12398, 29890,  1968, 20989,
         29901,  3713,  1815, 29954, 29871, 29906, 2

## Define Parameter for Text Generation

Each parameter influences the text generation in a specific way. Below are the parameters along with a brief explanation:

**`max_length`**:
* Sets the maximum number of tokens in the generated text (default is 50).
* Generation stops if the maximum length is reached before the model produces an EOS token.
* A higher `max_length` allows for longer generated texts but may increase the time and computational resources required.

**`min_length`**:
* Sets the minimum number of tokens in the generated text (default is 10).
* Generation continues until this minimum length is reached even if an EOS token is produced.

**`num_beams`**:
* In beam search, sets the number of "beams" or hypotheses to keep at each step (default is 4).
* A higher number of beams increases the chances of finding a good output but also increases the computational cost.

**`num_return_sequences`**:
* Specifies the number of independently computed sequences to return (default is 3).
* When using sampling, multiple different sequences are generated independently from each other.

**`early_stopping`**:
* Stops generation if the model produces the EOS (End Of Sentence) token, even if the predefined maximum length is not reached (default is True).
* Useful when an EOS token signifies the logical end of a text (often represented as `</s>`).

**`do_sample`**:
* Tokens are selected probabilistically based on their likelihood scores (default is True).
* Introduces randomness into the generation process for diverse outputs.
* The level of randomness is controlled by the 'temperature' parameter.

**`temperature`**:
* Adjusts the probability distribution used for sampling the next token (default is 0.7).
* Higher values make the generation more random, while lower values make it more deterministic.

**`top_k`**:
* Limits the number of tokens considered for sampling at each step to the top K most likely tokens (default is 50).
* Can make the generation process faster and more focused.

**`top_p`**:
* Also known as nucleus sampling, sets a cumulative probability threshold (default is 0.95).
* Tokens are sampled only from the smallest set whose cumulative probability exceeds this threshold.

**`repetition_penalty`**:
* Discourages the model from repeating the same token by modifying the token's score (default is 1.5).
* Values greater than 1.0 penalize repetitions, and values less than 1.0 encourage repetitions.


In [11]:
params = {'do_sample': True,
          #'early_stopping': True,
          #'num_beams': 5,
          #'num_return_sequences': 5,
          #'max_new_tokens': 128,
          #'min_new_tokens': 1,
          #'output_scores': True,
          # 'repetition_penalty': 1.0,
          'max_length': 8192,
          'temperature': 0.3,
          #'top_k': 50,
          'top_p': 0.95
          }

## Helper Functions

In [12]:
def compile_prompt(article, template, template_example, topic, topic_desciptions, examples, lang='German'):
    """ Compiles the prompt for the given article and model."""

    # Get the topic description and keywords
    topic_desciption = topic_desciptions[topic]
    topic_name = topic_desciption.get("name")
    topic_desc = topic_desciption.get("description")
    topic_keyw = topic_desciption.get("keywords")
    
    # Get the text of the article
    article_text = article.get("text")
    article_lang = article.get("lang")
    article_url = article.get("view_url")
    example_prompts = [template_example.format(text=example["text"], url=example["view_url"], label= "Ja" if example["label"] == 1 else "Nein") for example in examples]
    prompt = template.format(topic = topic_name, url = article_url,  topic_desc = topic_desc, topic_keyw = topic_keyw,
                             lang = article_lang, webpage_text=article_text, examples="".join(example_prompts))

    return prompt

In [13]:
article = {"view_url": "test.com", "text": "Lorem Ipsum", "lang": "de"}
exmaple_prompt = compile_prompt(article, PROMPT_TEMPLATE_FEW_SHOT, PROMPT_TEMPLATE_EXAMPLES, "cannabis", topic_desciptions, example_list, lang='German')
print(exmaple_prompt)

Klassifiziere die folgende Webseite auf de als themenbezogen oder nicht themenbezogen. Enthält er Informationen über 'die Legalisierung von Cannabis'? Bitte antworte nur mit 'Ja' oder 'Nein'.

Themenbeschreibung: Das CanG 2023 (Cannabisgesetz) wird den privaten Anbau von Cannabis durch Erwachsene für den persönlichen Gebrauch und den kollektiven nicht-kommerziellen Anbau legalisieren.
Themen-Schlüsselwörter: ['cannabis', 'canabis', 'cannabic', 'gras', 'cbd', 'droge', 'hanf', 'thc', 'canbe', 'legal', 'legalisierung', 'gesetz', 'verein', 'entkriminali']

Beispiele:

URL: '''google.de'''
Text: '''Cannabis ist eine Droge.'''
Antwort: 'Nein'

URL: '''example.com'''
Text: '''Katzen sind Tiere.'''
Antwort: 'Nein'


-------


Webseite :
URL: '''test.com'''
Text: '''Lorem Ipsum'''

Enthält diese Webseite relevante Informationen über 'die Legalisierung von Cannabis'?



In [14]:
def calculate_input_length(prompt):
    """ Calculates the length of the input sequence for the model. """

    # Tokenize the prompt
    tokenized_prompt = tokenizer(
        prompt, return_tensors="pt", add_special_tokens=False, truncation=False, padding=False)

    # Calculate the length of the input sequence
    input_length = tokenized_prompt.input_ids.size(1)

    return input_length

In [15]:
def generate_answers(model, tokenizer, prompt, params, remove_input=True):
    """Generates answers from a language model for a given prompt."""

    # Encode the prompt and generate the answers
    # prompt = convert_prompt_to_chat(tokenizer, SYTEM_PROMPT_TEMPLATE, prompt)
    # encoded_input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    
    # messages = [
    #     {"role": "system", "content": SYTEM_PROMPT_TEMPLATE},
    #     {"role": "user", "content": prompt}
    # ]
    # encoded_input = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    
    prompt_template = "<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n<|im_start|>assistant\n"
    
    prompt = prompt_template.format(system_prompt=SYTEM_PROMPT_TEMPLATE, user_prompt=prompt)
    encoded_input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

    if encoded_input.size()[1] > tokenizer.model_max_length:
        print("Input too long, truncating.")
        # encoded_input = encoded_input[:, :tokenizer.model_max_length]

    generated_outputs = model.generate(encoded_input, pad_token_id=tokenizer.eos_token_id, **params)
    
    #print("Generated outputs:", tokenizer.decode(generated_outputs[0], skip_special_tokens=True))

    # Decode and clean outputs
    outputs = []
    input_ids = encoded_input[0].tolist()

    for output in generated_outputs:
        output_ids = output.tolist()

        # Find the position where the input ends in the output
        input_length = len(input_ids)
        output_length = len(output_ids)
        
        # Ensure we don't go out of bounds
        if input_length < output_length and output_ids[:input_length] == input_ids:
            cleaned_text_ids = output_ids[input_length:]
        else:
            cleaned_text_ids = output_ids
        
        # Decode the cleaned output
        cleaned_text = tokenizer.decode(cleaned_text_ids, skip_special_tokens=True).strip()
        outputs.append(cleaned_text)

    return outputs

In [16]:
def parse_response(output_text):
    """Determines if the model's output signifies "Yes" (1) or "No" (0)."""
    text = output_text.lower()
    return 1 if "ja" in text else 0 if "nein" in text else 0

In [17]:
def calc_metrics(labels, preds):
    """
    Calculates the accuracy, precision, recall, and F1 score for the given labels and predictions and returns them in a dictionary.
    """

    metrics = {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='binary'),
        'recall': recall_score(labels, preds, average='binary'),
        'f1': f1_score(labels, preds, average='binary'),
    }

    return metrics

In [18]:
def sample_random_from_dataset(dataset, n=5, subset='test'):
    """
    Samples n random examples from a specified subset of the dataset.
    """
    n = min(n, len(dataset[subset]))
    random_indices = random.sample(range(len(dataset[subset])), n)
    sampled_dataset = dataset[subset].select(random_indices)
    return sampled_dataset

In [19]:
def load_model_and_tokenizer(model_details):
    """
    Loads a model and its corresponding tokenizer based on the provided model details.
    """
    model_name = model_details['model']
    tokenizer_class = model_details['tokenizer_class']
    model_class = model_details['model_class']
    
    # Cohere models and FLAN models
    if tokenizer_class == "AutoTokenizer" and model_class == "AutoModelForSeq2SeqLM":
        from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name, device_map="auto", load_in_8bit=True)
        
    # Vicuna models
    elif tokenizer_class == "LlamaTokenizer" and model_class == "LlamaForCausalLM":
        from transformers import LlamaTokenizer, LlamaForCausalLM
        tokenizer = LlamaTokenizer.from_pretrained(model_name)
        model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)
        
    #  LeoLM models  
    elif tokenizer_class == "AutoTokenizer" and model_class == "AutoModelForCausalLM":
        from transformers import AutoTokenizer, AutoModelForCausalLM
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True, trust_remote_code=False)
        
    else:
        raise ValueError("Model class not supported.")
        
    return tokenizer, model


**Different Sample Methods**

In [20]:
from random import sample


################### Random Sampling ############################

def sample_examples_random(dataset, k=2):
    """Samples k pairs of examples completely at random."""
    dataset_sampled = dataset.shuffle().select(range(k))
    return [example for example in dataset_sampled]


################### Random Sampling balanced ###################

def sample_examples_random_balanced(dataset, k=2):
    """Samples k pairs of examples, each pair containing one positive and one negative example."""
    # Separate the dataset into positive and negative examples
    positive_examples = [example for example in dataset if example['label'] == 1]
    negative_examples = [example for example in dataset if example['label'] == 0]
    
    # Sample k examples from each subset
    sampled_positive = sample(positive_examples, k)
    sampled_negative = sample(negative_examples, k)
    
    # Alternate between positive and negative examples to create pairs
    examples = []
    for idx in range(k):
        if idx % 2 == 0:
            examples.append(sampled_positive[idx])
        else:
            examples.append(sampled_negative[idx])
    
    return examples


################### KNN Sampling ############################

def sample_examples_knn(model, index, query, dataset, k=2):
    inferred_vector = model.encode(query, convert_to_tensor=True, show_progress_bar = False)
    sims = index.get_nns_by_vector(inferred_vector, k, search_k=-1, include_distances=False)
    return [dataset[idx] for idx in sims]

################### Expert Sampling ############################

# def sample_from_expert(curated_examples, topic, k=2):
    
#     curated_examples_topic = curated_examples[topic]
#     sampled_positive = sample(curated_examples_topic['positive_examples'], k)
#     sampled_negative = sample(curated_examples_topic['negative_examples'], k)
    
#     # Alternate between positive and negative examples to create pairs
#     examples = []
#     for idx in range(k):
#         if idx % 2 == 0:
#             examples.append(sampled_positive[idx])
#         else:
#             examples.append(sampled_negative[idx])
    
#     return examples


## Load Encoder for KNN Sampling

In [21]:
from annoy import AnnoyIndex

In [22]:
# Load the transformer-based model
from sentence_transformers import SentenceTransformer, util
encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda:0')

In [23]:
# Function to encode texts to embeddings
def encode_to_embedding(example):
    example['embeddings'] = encoder.encode(example['text'])
    return example

In [24]:
from collections import Counter

def majority_voting(answers):
    """Apply majority voting to a list of arbitrary classification answers."""
    count = Counter(answers)
    most_common = count.most_common(2)  # Get the two most common answers
    
    if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
        return "Tie"
    return most_common[0][0]

# Example usage with arbitrary labels
answers = ["Apple", "Banana", "Apple", "Orange"]
print(f"The majority vote is: {majority_voting(answers)}")

The majority vote is: Apple


**Get samples:**

In [25]:
#k = 2
# examples = sample_examples_random(dataset['train'], k) 
# examples = sample_examples_random_balanced(dataset['train'], k)
# examples = sample_examples_knn(dataset['train'], k)
# examples = sample_from_expert(curated_examples, topic, k)
# sample_examples_knn(model, article_index, dataset["train"][0]["text"], dataset["train"], k)
#print("Examples: ", examples)

## Generate Answers

In [26]:
from collections import defaultdict
import gc

In [27]:
eval_results = defaultdict(dict)

for model_details in MODELS: #-------------------------------------------------------------

    # Load model
    model_name = model_details['model']
    print(f"Loading model {model_name}")
    tokenizer, model = load_model_and_tokenizer(model_details)
    
    for topic in TOPICS:  # ---------------------------------------------------------------

        # Load dataset
        print(f"Loading dataset for {topic}")
        dataset = load_from_disk(f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}")
        #dataset['test'] = dataset['test'].filter(lambda x: x['token_count'] > 200)
        #dataset[SPLIT] = sample_random_from_dataset(dataset, n=10, subset=SPLIT)
        #print(dataset['test'][0])

        # Few-Shot ------------------------------------------------------------------------

        # Random Sampling
        if DEMONSTR_SAMPLING == "random":
            examples = sample_examples_random(dataset['train'], k=K) 
            #print("Examples: ", len(examples))  

        # Random Sampling (Balanced by classes)
        elif DEMONSTR_SAMPLING ==  "random_balanced":
            examples = sample_examples_random_balanced(dataset['train'], k=K)

        # KNN Cluster-based sampling
        elif DEMONSTR_SAMPLING == "knn":
            article_index = AnnoyIndex(encoder.get_sentence_embedding_dimension(), "angular")
            article_index.load(f'../../data/indices/page_index_{topic}.ann')
            
        # # Use curated examples
        # elif DEMONSTR_SAMPLING == "expert":
        #     examples = sample_from_expert(curated_examples, topic, k)

        # Zero-Shot ------------------------------------------------------------------------
        
        elif K == 0:
            examples = []
        else: 
            examples = []
    
        # Iterate over pages in test split
        answers = [] 
        for row in tqdm(dataset[SPLIT]): # ---------------------------------------------------
    
            # Dynamic Sampling 
            if DEMONSTR_SAMPLING == "knn" and K > 0:
                examples = sample_examples_knn(encoder, article_index, row["text"], dataset["train"], K)
    
            # Generate answers
            prompt_template = PROMPT_TEMPLATE_FEW_SHOT if K > 0 else PROMPT_TEMPLATE_ZERO_SHOT
            prompt = compile_prompt(row, prompt_template, PROMPT_TEMPLATE_EXAMPLES, topic, topic_desciptions, examples)
            answer = generate_answers(model, tokenizer, prompt, params)
            #print("Label:", row["label"])
            #print("Answers:", answer)
            answers.append(answer)

    
        # Add answers to the dataset
        dataset[SPLIT] = dataset[SPLIT].add_column("answers", answers)
        dataset.save_to_disk(f"../../data/tmp/processed_dataset_{topic}_answers_{K}s_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}_s_{model_name.split('/')[1]}_{FEATURES}_{SPLIT}_{DEMONSTR_SAMPLING}")
        
        # Calculate metrics
        print("Answers:", answers)
        answers_after_voting = [majority_voting(ans) for ans in answers]
        #print("Answers after voting:", answers_after_voting)
        answers_parsed = [parse_response(ans) for ans in answers_after_voting]
        metrics = calc_metrics(dataset[SPLIT]['label'], answers_parsed)
        eval_results[model_name][topic] = metrics
        print(f"Metrics for {model_name}: {metrics}")
        
    # Clear GPU memory to avoid memory errors
    model.cpu()
    torch.cuda.empty_cache()
    del model, tokenizer
    gc.collect()  # Explicitly invoking garbage collection
    torch.cuda.empty_cache()  # Clear cache again after garbage collection
    time.sleep(5)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model LeoLM/leo-mistral-hessianai-7b-chat


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.98s/it]


Loading dataset for cannabis


Saving the dataset (1/1 shards): 100%|██████████| 224737/224737 [00:00<00:00, 620211.83 examples/s]


Answers: [['Nein, die Webseite enthält keine relevanten Informationen über die Legalisierung von Cannabis.'], ['Nein, die Webseite enthält keine relevanten Informationen über die Legalisierung von Cannabis.'], ['Nein.'], ["Nein, die Webseite enthält keine relevanten Informationen über 'die Legalisierung von Cannabis'."], ['Nein, die gegebene Webseite enthält keine Informationen über die Legalisierung von Cannabis.'], ['Nein, die Webseite enthält keine relevanten Informationen über die Legalisierung von Cannabis.'], ['Nein, die Webseite enthält keine relevanten Informationen über die Legalisierung von Cannabis.'], ['Nein.'], ['Nein.'], ['Nein.'], ['Nein, die Webseite enthält keine Informationen über die Legalisierung von Cannabis.'], ['Nein.'], ["Nein, diese Webseite enthält keine relevanten Informationen über 'die Legalisierung von Cannabis'."], ['Nein.'], ['Nein, die Webseite enthält keine relevanten Informationen über die Legalisierung von Cannabis.'], ['Nein.'], ['Nein.'], ['Nein, d

Saving the dataset (1/1 shards): 100%|██████████| 229661/229661 [00:00<00:00, 600516.09 examples/s]


Answers: [["Nein, die gegebene Webseite enthält keine Informationen über 'erneuerbare Energiequellen'."], ['Nein, die Webseite enthält keine relevanten Informationen über erneuerbare Energiequellen.'], ['Nein, die Webseite enthält keine relevanten Informationen über erneuerbare Energiequellen.'], ['Nein.'], ['Nein, die Webseite enthält keine Informationen über erneuerbare Energiequellen.'], ["Nein, die Webseite enthält keine Informationen über 'erneuerbare Energiequellen'."], ['Nein, die Webseite enthält keine Informationen über erneuerbare Energiequellen.'], ["Nein, die Webseite enthält keine Informationen über 'erneuerbare Energiequellen'."], ["Nein, die Webseite enthält keine Informationen über 'erneuerbare Energiequellen'."], ["Nein, die Webseite enthält keine Informationen über 'erneuerbare Energiequellen'."], ["Nein, die Webseite enthält keine relevanten Informationen über 'erneuerbare Energiequellen'."], ['Nein'], ['Nein, die Webseite enthält keine Informationen über erneuerbare

Saving the dataset (1/1 shards): 100%|██████████| 266322/266322 [00:00<00:00, 601296.02 examples/s]


Answers: [["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, diese Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, diese Webseite enthält keine relevanten Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["Nein, die Webseite enthält keine Informationen über 'Kindergrundsicherung'."], ["N

In [28]:
dataset['test'][0]

{'_id': '648c2adb8e8cadbd29095617',
 'batch_id': 15,
 'domain': 'schule.at',
 'view_url': 'www.schule.at/gewinnspiel-buch-tori-twister-stuermisch-unterwegs',
 'lang': 'de',
 'text': 'Your browser does not support JavaScript! Webmail Bildungs-News Schul-Themen Unterrichts-Portale Schulführer Termine Tools & Apps Webmail schule.at Gewinnspiel: Buch "Tori Twister - Stürmisch unterwegs" © KOSMOS Verlag Tori Twister - Stürmisch unterwegs Die Autorin von "Tori Twister - Stürmisch unterwegs" verpackt Naturwissen zu Wetterphänomenen und Umweltschutz in eine spannende und lustige Geschichte. Wir haben bei Autorin Marikka Pfeiffer nachgefragt, was Ihre Beweggründe waren, dieses Buch zu schreiben. Und mit etwas Glück können Sie bei unserem Gewinnspiel ein Buchexemplar gewinnen . Als Geschichtenerzählerin liegt es mir sehr am Herzen, mit meinen Büchern die Freude am Lesen zu wecken, denn die Vorstellungskraft und das Lesen sind für mich zwei der wichtigsten Kulturgüter, die wir haben. Daher ist di

## Save and Output Results

**Get chunk level predictions:**

In [29]:
from tabulate import tabulate

In [30]:
# Define the file path to save the dictionary
file_path = f"eval_results_icl_{K}s_{DEMONSTR_SAMPLING}_{SPLIT}_chat_chunks.json"

In [31]:
import json

# Save the dictionary to disk as JSON
with open(file_path, "w") as file:
    json.dump(eval_results, file)

In [32]:
import json

# Load the dictionary from the JSON file
with open(file_path, "r") as file:
    eval_results = json.load(file)

In [33]:
# Identify all topics (assuming all models are evaluated on the same topics)
topics = list(next(iter(eval_results.values())).keys())

# Prepare headers for the table: each topic will have four metrics
headers = ["Model"] + \
    [f"{topic} {metric}" for topic in topics for metric in [
        "Acc.", "Prec.", "Rec.", "F1"]]

# Prepare rows: one row per model, containing metrics for each topic
rows = []
for model_name_t, topics_metrics in eval_results.items():
    row = [model_name_t]  # Start with the model name
    for topic in topics:
        metrics = topics_metrics.get(topic, {})
        row.extend([metrics.get('accuracy', 0.0), metrics.get(
            'precision', 0.0), metrics.get('recall', 0.0), metrics.get('f1', 0.0)])
    rows.append(row)

# Generate the HTML table
table_html = tabulate(rows, headers=headers, tablefmt="html",
                      showindex="never", floatfmt=".3f")

In [34]:
from IPython.display import display, HTML
display(HTML(table_html))

Model,cannabis Acc.,cannabis Prec.,cannabis Rec.,cannabis F1,energie Acc.,energie Prec.,energie Rec.,energie F1,kinder Acc.,kinder Prec.,kinder Rec.,kinder F1
LeoLM/leo-mistral-hessianai-7b-chat,0.787,0.991,0.679,0.806,0.929,0.987,0.856,0.917,0.915,0.984,0.882,0.930


**Get page level predictions:**

In [35]:
from collections import Counter

In [36]:
def majority_voting(answers):
    """Apply majority voting to a list of arbitrary classification answers."""
    count = Counter(answers)
    most_common = count.most_common()  # Get all common answers sorted by frequency

    if not most_common:
        return 0 # Handle empty input scenario

    # Check for ties at the highest count
    max_votes = most_common[0][1]
    tied_classes = [cls for cls, votes in most_common if votes == max_votes]

    if len(tied_classes) > 1:
        return max(tied_classes)  # Return the maximum class label in case of a tie
    return tied_classes[0]  # Return the class with the most votes

majority_voting([1, 1, 2, 2, 2, 3])

2

In [37]:
from collections import defaultdict
eval_results_pages = defaultdict(dict)

for topic in TOPICS: # ----------------------------------------------------------------------
    for model_details in MODELS: # -------------------------------------------------------------
        
        model_name = model_details['model']
            
        print(f"\n\n###### Evaluating model {model_name} on {topic} ###### \n\n")
        dataset = load_from_disk(f"../../data/tmp/processed_dataset_{topic}_answers_{K}s_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}_s_{model_name.split('/')[1]}_{FEATURES}_{SPLIT}_{DEMONSTR_SAMPLING}")
        
        #print(dataset)
        
        # Group dataset examples by URL, with a fallback to domain
        grouped_dataset = {}
        for example in tqdm(dataset[SPLIT]):
            url = example.get("view_url") or example.get("domain")
            example_filtered = {k: example[k] for k in ["text", "domain", "answers", "label", "category", "annotation_type", "lang"]}
            grouped_dataset.setdefault(url, []).append(example_filtered)
            
        # Extract labels
        labels = []
        for url, chunks in grouped_dataset.items():
            label = [chunk["label"] for chunk in chunks]
            labels.append(max(label))
            
        # Merge chunk level predictions
        predictions = []
        for url, chunks in grouped_dataset.items():
            
            for chunk in chunks:
                answers = [parse_response(ans) for ans in chunk["answers"]]
                chunk["pred"] = majority_voting(answers)
                
            preds = [chunk["pred"] for chunk in chunks]
            pred = majority_voting([pred for pred in preds if pred > 0]) if max(preds) > 0 else 0
            predictions.append(pred)

        # Use the trained model to make predictions on the test set
        print(predictions)
        print(labels)
        metrics = calc_metrics(labels, predictions)
        print(f"Metrics for {model_name} on {topic}: {metrics}")
        
        # Update the eval_results dictionary
        eval_results_pages[model_name][topic] = metrics
    



###### Evaluating model LeoLM/leo-mistral-hessianai-7b-chat on cannabis ###### 




100%|██████████| 507/507 [00:00<00:00, 8964.04it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Metrics for LeoLM/leo-mistral-hessianai-7b-chat on cannabis: {'accuracy': 0.9534883720930233, 'precision': 0.9090909090909091, 'recall': 1.0, 'f1': 0.9523809523809523}


###### Evaluating model LeoLM/leo-mistral-hessianai-7b-chat on energie ###### 




100%|██████████| 579/579 [00:00<00:00, 9001.12it/s]


[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Metrics for LeoLM/leo-mistral-hessianai-7b-chat on energie: {'accuracy': 0.9565217391304348, 'precision': 0.92, 'recall': 1.0, 'f1': 0.9583333333333334}


###### Evaluating model LeoLM/leo-mistral-hessianai-7b-chat on kinder ###### 




100%|██████████| 316/316 [00:00<00:00, 8888.86it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Metrics for LeoLM/leo-mistral-hessianai-7b-chat on kinder: {'accuracy': 0.9302325581395349, 'precision': 0.9090909090909091, 'recall': 0.9523809523809523, 'f1': 0.9302325581395349}


In [38]:
# Define the file path to save the dictionary
file_path = f"eval_results_icl_{K}s_{DEMONSTR_SAMPLING}_{SPLIT}_chat_pages.json"

In [39]:
# Save the dictionary to disk as JSON
with open(file_path, "w") as file:
    json.dump(eval_results_pages, file)

In [40]:
with open(file_path, "r") as file:
    eval_results_pages = json.load(file)

In [41]:
# Identify all topics (assuming all models are evaluated on the same topics)
topics = list(next(iter(eval_results_pages.values())).keys())

# Prepare headers for the table: each topic will have four metrics
headers = ["Model"] + [f"{topic} {metric}" for topic in topics for metric in ["Acc.", "Prec.", "Rec.", "F1"]]

# Prepare rows: one row per model, containing metrics for each topic
rows = []
for model, topics_metrics in eval_results_pages.items():
    row = [model]  # Start with the model name
    for topic in topics:
        metrics = topics_metrics.get(topic, {})
        row.extend([metrics.get('accuracy',0.0), metrics.get('precision',0.0), metrics.get('recall',0.0), metrics.get('f1',0.0)])
    rows.append(row)

# Generate the HTML table
table_html = tabulate(rows, headers=headers, tablefmt="html", showindex="never", floatfmt=".3f")

In [42]:
display(HTML(table_html))

Model,cannabis Acc.,cannabis Prec.,cannabis Rec.,cannabis F1,energie Acc.,energie Prec.,energie Rec.,energie F1,kinder Acc.,kinder Prec.,kinder Rec.,kinder F1
LeoLM/leo-mistral-hessianai-7b-chat,0.953,0.909,1.000,0.952,0.957,0.920,1.000,0.958,0.930,0.909,0.952,0.930


In [43]:
len(grouped_dataset.keys())

43